# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   -- ------------------------------------- 1.0/19.3 MB 7.2 MB/s eta 0:00:03
   ------ --------------------------------- 3.1/19.3 MB 8.8 MB/s eta 0:00:02
   --------------- ------------------------ 7.3/19.3 MB 12.9 MB/s eta 0:00:01
   --------------------------- ------------ 13.4/19.3 MB 17.1 MB/s eta 0:00:01
   ---------------------------------------- 19.3/19.3 MB 21.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 35.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [7]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_TEST_KEY_KdR')

# test if the key is loaded correctly
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY is not set. Please set it in your environment variables.")


In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [36]:

from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.5,
)

In [9]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [20]:
import os

if not os.path.exists(doc_path):
	raise FileNotFoundError(f"File not found: {doc_path}")

loader = TextLoader(doc_path, encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(api_key = OPENAI_API_KEY)
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state_of_the_union.txt")

In [21]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [22]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [23]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [24]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [25]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [26]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [27]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\karel\AppData\Local\Temp\ipykernel_18292\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
from langchain_openai import ChatOpenAI
from openai import OpenAI 


# Use a completion model for the agent

agent_completion = initialize_agent(
    tools, llm_completion, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

client = OpenAI( api_key=OPENAI_API_KEY)

prompt = "What did biden say about ketanji brown jackson in the state of the union address?"

# Complete the prompt using the agent

    



Response(id='resp_6852c76f07fc8191b0859f1c1b34844202b7b3ff75ac4dbd', created_at=1750255471.0, error=None, incomplete_details=None, instructions='You are a helpful assistant.', metadata={}, model='gpt-3.5-turbo-0125', object='response', output=[ResponseOutputMessage(id='msg_6852c76f4fe48191a82824dae8a79e7002b7b3ff75ac4dbd', content=[ResponseOutputText(annotations=[], text='In his State of the Union address on March 1, 2022, President Joe Biden praised Judge Ketanji Brown Jackson, his nominee for the Supreme Court, highlighting her qualifications, experience, and dedication to upholding the rule of law. Biden called her a "brilliant legal mind with the utmost character and integrity" and reiterated his belief that she will serve the nation with distinction on the Supreme Court if confirmed.', type='output_text', logprobs=None)], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, max

In [67]:
prompt = "Why use ruff over flake?"

response = client.responses.create(
    model="gpt-3.5-turbo",
    instructions = "You are a helpful assistant.",
    input = prompt
)

print(response)
print(response)


Response(id='resp_6852c7f3d008819f83971b34e004ac7d0470883a8f0ea148', created_at=1750255603.0, error=None, incomplete_details=None, instructions='You are a helpful assistant.', metadata={}, model='gpt-3.5-turbo-0125', object='response', output=[ResponseOutputMessage(id='msg_6852c7f44d68819fad3858d3e7fc25100470883a8f0ea148', content=[ResponseOutputText(annotations=[], text='It seems like you might be referring to two different types of food for pets, such as dogs or cats. Without more context, it\'s difficult to provide a specific answer. \n\nIf you are talking about dog food, "ruff" and "flake" are not standard terms related to dog food. Generally, it\'s essential to choose a high-quality dog food that meets your pet\'s nutritional needs based on factors like their age, size, breed, and health condition. \n\nIf you could provide more information or clarification, I would be happy to help further.', type='output_text', logprobs=None)], role='assistant', status='completed', type='message'

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [68]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [46]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [48]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question
Action: State of Union QA System
Action Input: What did biden say about ketanji brown jackson in the state of the union address?
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [50]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [53]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [55]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [57]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the State of Union QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I should use the State of Union QA System to answer the second question.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: I now know the final answer.
Final Answer: No, the president did not mention a specific tool in the state of the union.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, the president did not mention a specific tool in the state of the union.'}